<a href="https://colab.research.google.com/github/JonasHagstrom79/Natural-Language-Processing/blob/main/Pipeline_Masked_Language_Modelling(Article_Spinner).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv


--2023-02-26 15:18:51--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  5.51MB/s    in 0.9s    

2023-02-26 15:18:53 (5.51 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [2]:
# Install transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.9 MB/s eta 0:00:00


In [3]:
# Import the needed tools for the notebook
import numpy as np
import pandas as pd
import textwrap
from pprint import pprint

from transformers import pipeline

In [4]:
# Load in the data
df = pd.read_csv('bbc_text_cls.csv')

In [5]:
# Inspect the dataframe
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [7]:
# Print out the uniqe set of labels to see what exists
labels = set(df['labels'])
labels

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [8]:
# Pick a label
label = 'business'

In [11]:
# Filter out any rows that not match our label, and to grab just the column containing our text
texts = df[df['labels'] == label]['text']
texts.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

In [13]:
# Set a random seed so that you can replicat these results for yourself
np.random.seed(1234)

In [14]:
# Randomly choose a document, the result is called doc
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]

In [15]:
# Print the choosen article to see what it looks like
print(textwrap.fill(doc, replace_whitespace=False, fix_sentence_endings=True))

Bombardier chief to leave company

Shares in train and plane-making
giant Bombardier have fallen to a 10-year low following the departure
of its chief executive and two members of the board.

Paul Tellier,
who was also Bombardier's president, left the company amid an ongoing
restructuring.  Laurent Beaudoin, part of the family that controls the
Montreal-based firm, will take on the role of CEO under a newly
created management structure.  Analysts said the resignations seem to
have stemmed from a boardroom dispute.  Under Mr Tellier's tenure at
the company, which began in January 2003, plans to cut the worldwide
workforce of 75,000 by almost a third by 2006 were announced.  The
firm's snowmobile division and defence services unit were also sold
and Bombardier started the development of a new aircraft seating 110
to 135 passengers.

Mr Tellier had indicated he wanted to stay at the
world's top train maker and third largest manufacturer of civil
aircraft until the restructuring was comple

In [16]:
# Insanciate our pipeline object, define fill-mask to defin the task. The model is Roberta wich is a derivative of Bert
mlm = pipeline('fill-mask')

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [17]:
# Test the model, pass in the title of choosen article
mlm('Bombardier chief to leave <mask>')

[{'score': 0.0695078894495964,
  'token': 633,
  'token_str': ' job',
  'sequence': 'Bombardier chief to leave job'},
 {'score': 0.06693075597286224,
  'token': 1470,
  'token_str': ' France',
  'sequence': 'Bombardier chief to leave France'},
 {'score': 0.0527353398501873,
  'token': 558,
  'token_str': ' office',
  'sequence': 'Bombardier chief to leave office'},
 {'score': 0.025823084637522697,
  'token': 2201,
  'token_str': ' Paris',
  'sequence': 'Bombardier chief to leave Paris'},
 {'score': 0.02136855758726597,
  'token': 896,
  'token_str': ' Canada',
  'sequence': 'Bombardier chief to leave Canada'}]

In [20]:
# Try the model on the next scetence in the article
text = 'Shares in <mask> and plane-making ' + \
  'gigant Bombardier have fallen to a 10-year low following the departure ' + \
  'of its chief executive and two members of the board.'

mlm(text)


[{'score': 0.4918804466724396,
  'token': 11016,
  'token_str': ' Airbus',
  'sequence': 'Shares in Airbus and plane-making gigant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of the board.'},
 {'score': 0.16927364468574524,
  'token': 15064,
  'token_str': ' aerospace',
  'sequence': 'Shares in aerospace and plane-making gigant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of the board.'},
 {'score': 0.12086879462003708,
  'token': 6722,
  'token_str': ' Boeing',
  'sequence': 'Shares in Boeing and plane-making gigant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of the board.'},
 {'score': 0.04237903282046318,
  'token': 8537,
  'token_str': ' airlines',
  'sequence': 'Shares in airlines and plane-making gigant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of

In [21]:
# Try the model on the scentence
text = 'Shares in train and plane-making ' + \
  'gigant Bombardier have fallen to a 10-year low following the <mask> ' + \
  'of its chief executive and two members of the board.'

pprint(mlm(text))

[{'score': 0.5020133256912231,
  'sequence': 'Shares in train and plane-making gigant Bombardier have fallen '
              'to a 10-year low following the resignation of its chief '
              'executive and two members of the board.',
  'token': 6985,
  'token_str': ' resignation'},
 {'score': 0.24936453998088837,
  'sequence': 'Shares in train and plane-making gigant Bombardier have fallen '
              'to a 10-year low following the departure of its chief executive '
              'and two members of the board.',
  'token': 5824,
  'token_str': ' departure'},
 {'score': 0.14369048178195953,
  'sequence': 'Shares in train and plane-making gigant Bombardier have fallen '
              'to a 10-year low following the departures of its chief '
              'executive and two members of the board.',
  'token': 25624,
  'token_str': ' departures'},
 {'score': 0.0362585075199604,
  'sequence': 'Shares in train and plane-making gigant Bombardier have fallen '
              'to a 10

In [22]:
# Try the model on the scentence
text = 'Shares in train and plane-making ' + \
  'gigant Bombardier hav fallen to a 10-year low following the departure ' + \
  'of its chief <mask> and two members of the board.'

pprint(mlm(text))

[{'score': 0.9872985482215881,
  'sequence': 'Shares in train and plane-making gigant Bombardier hav fallen '
              'to a 10-year low following the departure of its chief executive '
              'and two members of the board.',
  'token': 1031,
  'token_str': ' executive'},
 {'score': 0.006151681300252676,
  'sequence': 'Shares in train and plane-making gigant Bombardier hav fallen '
              'to a 10-year low following the departure of its chief '
              'executives and two members of the board.',
  'token': 4585,
  'token_str': ' executives'},
 {'score': 0.0015382386045530438,
  'sequence': 'Shares in train and plane-making gigant Bombardier hav fallen '
              'to a 10-year low following the departure of its chief economist '
              'and two members of the board.',
  'token': 7473,
  'token_str': ' economist'},
 {'score': 0.001493952120654285,
  'sequence': 'Shares in train and plane-making gigant Bombardier hav fallen '
              'to a 10-yea

In [23]:
# Try the model on the scentence
text = 'Shares in train and plane-making ' + \
  'gigant Bombardier hav fallen to a 10-year low following the departure ' + \
  'of its chief executive and two <mask> of the board.'

pprint(mlm(text))

[{'score': 0.9405950903892517,
  'sequence': 'Shares in train and plane-making gigant Bombardier hav fallen '
              'to a 10-year low following the departure of its chief executive '
              'and two members of the board.',
  'token': 453,
  'token_str': ' members'},
 {'score': 0.037334002554416656,
  'sequence': 'Shares in train and plane-making gigant Bombardier hav fallen '
              'to a 10-year low following the departure of its chief executive '
              'and two thirds of the board.',
  'token': 29193,
  'token_str': ' thirds'},
 {'score': 0.008695954456925392,
  'sequence': 'Shares in train and plane-making gigant Bombardier hav fallen '
              'to a 10-year low following the departure of its chief executive '
              'and two directors of the board.',
  'token': 5392,
  'token_str': ' directors'},
 {'score': 0.003251635702326894,
  'sequence': 'Shares in train and plane-making gigant Bombardier hav fallen '
              'to a 10-year low f

In [26]:
# use the TfidfVectorizer class from scikit-learn to calculate the TF-IDF scores for each word in the document.
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
def mask_words_with_highest_tfidf(text, n=3, mask_char='*'):
    # Create a TF-IDF vectorizer object
    vectorizer = TfidfVectorizer(stop_words='english')
    
    # Fit and transform the input text using the vectorizer
    X = vectorizer.fit_transform([text])
    
    # Create a DataFrame to store the words and their corresponding TF-IDF scores
    df = pd.DataFrame(X.T.todense(), index=vectorizer.get_feature_names(), columns=["tfidf"])
    df = df.sort_values('tfidf', ascending=False)
    
    # Select the top N words with the highest TF-IDF scores
    top_words = list(df.index[:n])
    
    # Replace the selected words with the mask character
    masked_text = text
    for word in top_words:
        masked_text = masked_text.replace(word, mask_char * len(word))
    
    return masked_text

In [28]:
# Load in the data
df = pd.read_csv('bbc_text_cls.csv')

# Select a document to process
document = df['text'][0]

# Call the function to mask and replace the top 3 words with the highest TF-IDF scores
masked_document = mask_words_with_highest_tfidf(document, n=3, mask_char='*')

# Print the original and masked documents side by side
print('=' * 80)
print('Original Document:')
print('-' * 80)
print(textwrap.fill(document, width=80))
print('=' * 80)
print('Masked Document:')
print('-' * 80)
print(textwrap.fill(masked_document, width=80))
print('=' * 80)


Original Document:
--------------------------------------------------------------------------------
Ad sales boost Time Warner profit  Quarterly profits at US media giant
TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from
$639m year-earlier.  The firm, which is now one of the biggest investors in
Google, benefited from sales of high-speed internet connections and higher
advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from
$10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at
Warner Bros, and less users for AOL.  Time Warner said on Friday that it now
owns 8% of search-engine Google. But its own internet business, AOL, had has
mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were
lower than in the preceding three quarters. However, the company said AOL's
underlying profit before exceptional items rose 8% on the back of stronger
internet advertising revenues. It hopes to increase subscrib

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
